<a href="https://colab.research.google.com/github/Smpests/KeepLearning/blob/master/LearningWithChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 零基础用ChatGPT学机器学习

### 首先，我们需要安装TensorFlow。您可以在终端或命令提示符中运行以下命令来安装TensorFlow：

In [2]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 接下来，我们将定义模型的架构。我们将使用一个简单的循环神经网络（RNN）模型，其中包含一个嵌入层、一个LSTM层和一个密集层。

In [17]:
import tensorflow as tf
import numpy as np

class LanguageModel(tf.keras.Model):
    """
    定义语言对话模型的架构
    """
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(LanguageModel, self).__init__(self)
        # 定义模型的嵌入层
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # 定义模型的LSTM层
        self.lstm = tf.keras.layers.LSTM(rnn_units)
        # 定义模型的密集层
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs):
        """
        定义模型的前向传播
        """
        # 将输入数据传递给嵌入层，获取嵌入向量
        x = self.embedding(inputs)
        # 将嵌入向量传递给LSTM层
        x = self.lstm(x)
        # 将LSTM层的输出传递给密集层，得到预测的下一个词的概率分布
        x = self.dense(x)
        return x

class DataGenerator:
    def __init__(self, encoded_text, batch_size, seq_length):
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.total_size = batch_size * (seq_length + 1)
        self.encoded_text = encoded_text

        # 检查文本的长度是否足够生成一个完整的子序列
        if len(self.encoded_text) < self.total_size:
            raise ValueError(f"文本长度不足，至少需要 {self.total_size} 个字符。")

        # 我们将 encoded_text 列表分割成多个子序列，每个子序列的长度为 seq_length+1。
        # 其中前 seq_length 个元素作为输入序列，最后一个元素作为输出序列。
        self.num_batches = len(encoded_text) // self.total_size
        self.x_data = np.zeros((self.num_batches, self.batch_size, self.seq_length), dtype=np.int32)
        self.y_data = np.zeros((self.num_batches, self.batch_size), dtype=np.int32)
        for i in range(self.num_batches):
            start = i * self.total_size
            end = start + self.total_size
            batch_text = encoded_text[start:end]
            for j in range(self.batch_size):
                x = batch_text[j * self.seq_length:(j + 1) * self.seq_length]
                y = batch_text[(j + 1) * self.seq_length]
                self.x_data[i, j, :] = x
                self.y_data[i, j] = y

        # 保存最后一个子序列，用于生成预测
        self.last_x = np.zeros((self.batch_size, self.seq_length), dtype=np.int32)
        self.last_x[:] = self.x_data[-1, :, :]
        self.last_y = self.y_data[-1, :]

    def __len__(self):
        # 返回生成器的迭代次数
        return self.num_batches

    def __getitem__(self, idx):
        # 返回给定索引对应的批次
        return self.x_data[idx], self.y_data[idx]

    def get_last_batch(self):
        # 返回最后一个批次，用于生成预测
        return self.last_x, self.last_y

### 下一步是训练模型并使用它来生成新的文本。这里我们需要定义一些训练参数，包括学习率、批次大小、序列长度等等。

In [20]:
# 反复问ChatGPT依然跑不起来

# 定义训练参数
learning_rate = 0.01
batch_size = 128
seq_length = 100
epochs = 30

# 在实际应用中，训练数据可能非常大，可能需要从文件或数据库中读取。
# 这里为了简化示例，我们假设训练数据已经存在，并将其作为一个字符串传递给 DataGenerator 类。

text = "This is a sample text for testing purposes. It has a total length of 12928 characters which is enough to generate a batch of size 128 and sequence length 100. This is just a random string of text that is long enough to meet the requirements. We could use any text of sufficient length for this purpose."
# 定义数据生成器
data_generator = DataGenerator(text, batch_size, seq_length)

# 创建模型model = LanguageModel(data_generator.vocab_size, 256, 1024)


# 定义优化器和损失函数
optimizer = tf.keras.optimizers.Adam(learning_rate)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# 训练模型
for epoch in range(epochs):
    epoch_loss = 0
    for i in range(data_generator.num_batches):
        x_batch = data_generator.x_batches[i]
        y_batch = data_generator.y_batches[i]
        with tf.GradientTape() as tape:
            # 在前向传播期间记录梯度
            logits = model(x_batch)
            loss = loss_fn(y_batch, logits)
            # 计算梯度
            gradients = tape.gradient(loss, model.trainable_variables)
            # 根据梯度更新模型参数
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        epoch_loss += loss
    print(f"Epoch {epoch+1}: Loss={epoch_loss}")

ValueError: ignored

### 训练过程中，我们按批次提供数据，计算模型输出和损失，并使用反向传播算法更新模型参数。每个周期的损失都会被累加并打印出来。

完成训练后，我们可以使用模型生成新的文本：

In [ ]:
# 定义生成文本的参数
num_words = 100
start_seed = "hello"

# 将起始种子转换为数字序列
token_list = data_generator.tokenizer.texts_to_sequences([start_seed])[0]
# 在开始生成新文本之前，我们需要先通过模型预测下一个词
# 然后将其添加到种子中，并移除第一个词以维持种子的长度
for _ in range(num_words):
    x = np.array(token_list[-seq_length:]).reshape(1, seq_length)
    # 使用模型预测下一个词
    predicted = model(x).argmax(axis=1)[0]
    # 将新的词添加到种子中
    token_list.append(predicted)
# 将数字序列转换回文本
generated_text = data_generator.tokenizer.sequences_to_texts([token_list])[0]
print(generated_text)